In [1]:
import numpy as np
from scipy.linalg import qr, cholesky

def randomized_preconditioned_qr(M, d):
    m, n = M.shape
    assert d < m, "Sketch dimension d must be less than the number of rows m"

    # Step 1: Random Sketching
    S = np.random.randn(d, m)  # S ∈ ℝ^{d×m}
    Msk = S @ M                # Msk ∈ ℝ^{d×n}

    # Step 2: QR on Sketch
    Qsk, Rsk = qr(Msk, mode='economic')  # Rsk ∈ ℝ^{n×n}

    # Step 3: Preconditioning
    Rsk_inv = np.linalg.inv(Rsk)         # Rsk^{-1}
    Mpre = M @ Rsk_inv                   # Mpre ∈ ℝ^{m×n}

    # Step 4: CholeskyQR
    Gram = Mpre.T @ Mpre                 # Mpre^T Mpre
    Rpre = cholesky(Gram, lower=False)   # Rpre ∈ ℝ^{n×n}
    Q = Mpre @ np.linalg.inv(Rpre)       # Q ∈ ℝ^{m×n}

    # Step 5: Undo Preconditioning
    R = Rpre @ Rsk                       # Final R ∈ ℝ^{n×n}

    return Q, R


In [2]:
def cholesky_qr(M):
    m, n = M.shape
    assert m >= n, "CholeskyQR requires m ≥ n"

    # Step 1: Compute Gram matrix
    G = M.T @ M

    # Step 2: Cholesky decomposition
    R = cholesky(G, lower=False)

    # Step 3: Recover Q
    Q = M @ np.linalg.inv(R)

    return Q, R


In [3]:
np.random.seed(0)
M = np.random.randn(100, 10)

In [4]:
Q, R = randomized_preconditioned_qr(M, d=30)
print(np.allclose(Q @ R, M))  # Should print: True

True


In [5]:
U, _ = np.linalg.qr(np.random.randn(100, 10))
V, _ = np.linalg.qr(np.random.randn(10, 10))
singular_values = np.logspace(0, -10, 10)  # Rapid decay
M_bad = U @ np.diag(singular_values) @ V.T

In [6]:
randomized_preconditioned_qr(M_bad, d=30)

(array([[ 8.85502719e-02,  1.82380286e-01, -9.45190190e-02,
         -6.36470553e-02, -1.70222378e-01, -9.01992810e-02,
         -6.20032176e-02, -4.30888349e-02,  3.96980026e-02,
         -2.38406524e-01],
        [-5.41965930e-02, -6.42086481e-02,  2.21386411e-02,
          1.99193745e-01, -8.32306002e-02, -4.01098206e-02,
          1.22271302e-01,  1.03208553e-01, -9.50942195e-03,
          1.06622068e-01],
        [-1.79608722e-01,  3.40210969e-02,  1.57415054e-02,
         -4.99549094e-02,  2.32759237e-02, -1.80380725e-01,
          8.22208865e-02, -3.07543711e-02,  1.53366381e-01,
         -4.25550728e-02],
        [ 6.43783572e-02,  1.40192946e-01, -1.63825117e-01,
         -2.54704443e-02, -2.15588058e-01,  1.15622921e-02,
         -1.34025287e-01,  2.88549565e-02,  5.72364380e-02,
          5.20947303e-03],
        [ 6.35204812e-02, -1.09261666e-01, -3.56598249e-02,
          1.27964361e-02, -1.60870167e-01, -4.47079781e-02,
         -7.80589514e-02, -5.98476206e-02,  5.544862

In [9]:
cholesky_qr(M_bad) # It should give an error

LinAlgError: 9-th leading minor of the array is not positive definite

In [12]:
import time
import numpy as np
from scipy.sparse import random as sparse_random
from scipy.fft import fft
from scipy.linalg import qr, cholesky
from numpy.random import default_rng

rng = default_rng()

# -----------------------------
# Sketching Matrix Families
# -----------------------------

def gaussian_sketch(d, m):
    """Dense Gaussian sketching matrix."""
    S = rng.standard_normal((d, m))
    def apply(M):
        return S @ M
    return apply

def saso_sketch(d, m, density=0.01):
    """Sparse sketching matrix (SASO)."""
    S = sparse_random(d, m, density=density, format='csr', random_state=rng).toarray()
    def apply(M):
        return S @ M
    return apply

def srft_sketch(d, m):
    """Subsampled Randomized Fourier Transform (SRFT)."""
    # Random ±1 diagonal (as vector, not full matrix)
    D = rng.choice([-1, 1], size=m)
    # Random row indices
    indices = rng.choice(m, size=d, replace=False)

    def apply(M):
        # Step 1: random sign flips
        M_tilde = D[:, None] * M
        # Step 2: FFT across rows
        M_fft = fft(M_tilde, axis=0, norm="ortho")
        # Step 3: Subsample rows
        return np.sqrt(m / d) * M_fft[indices, :].real

    return apply

# -----------------------------
# CholeskyQR
# -----------------------------

def cholqr(M):
    """Basic CholeskyQR decomposition."""
    A = M.T @ M
    R = cholesky(A, lower=False)
    Q = M @ np.linalg.inv(R)
    return Q, R

# -----------------------------
# CQRRPT Core
# -----------------------------

def cqrrptcore(M, Msk):
    """Core CQRRPT algorithm."""
    Qsk, Rsk, J = qr(Msk, pivoting=True, mode='economic')
    k = np.linalg.matrix_rank(Rsk)
    Mk = M[:, J[:k]]
    Ask_k = Rsk[:k, :k]
    Mpre = Mk @ np.linalg.inv(Ask_k)
    Qk, Rpre = cholqr(Mpre)
    Rk = Rpre @ Rsk[:k, :]
    return Qk, Rk, J

# -----------------------------
# CQRRPT Master Function
# -----------------------------

def cqrrpt(M, sketch_type="gaussian", gamma=1.25, **kwargs):
    """Run CQRRPT with specified sketch type."""
    m, n = M.shape
    d = int(np.ceil(gamma * n))

    if sketch_type == "gaussian":
        S = gaussian_sketch(d, m)
    elif sketch_type == "saso":
        S = saso_sketch(d, m, **kwargs)
    elif sketch_type == "srft":
        S = srft_sketch(d, m)
    else:
        raise ValueError(f"Unknown sketch_type: {sketch_type}")

    Msk = S(M)
    return cqrrptcore(M, Msk)

# -----------------------------
# Timing Comparison
# -----------------------------

if __name__ == "__main__":
    m, n = 10000, 100
    M = rng.standard_normal((m, n))

    sketch_types = ['gaussian', 'saso', 'srft']
    for sketch in sketch_types:
        start = time.time()
        Q, R, J = cqrrpt(M, sketch_type=sketch)
        end = time.time()
        print(f"{sketch.upper()} sketch time: {end - start:.4f} seconds")
        print(f"{sketch.upper()} orthogonality error:",
              np.linalg.norm(Q.T @ Q - np.eye(Q.shape[1])))

GAUSSIAN sketch time: 0.1541 seconds
GAUSSIAN orthogonality error: 8.563333152144005e-15
SASO sketch time: 0.0878 seconds
SASO orthogonality error: 8.86028166054233e-15
SRFT sketch time: 0.1759 seconds
SRFT orthogonality error: 8.844126580966306e-15
